# Diff in Diff Regression by Clarissa

In [14]:
import pandas as pd

df = pd.read_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/big_merge.csv")
df_clean = df.dropna(axis=0).copy()
df.sample(3)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year
4538,210,210,male,Toba Tek Singh,rural,70,54,0.771429,2015
4672,344,344,female,Kohlu,rural,104,32,0.307692,2015
986,270,270,female,Jhelum,rural,282,255,0.904255,2006


In [15]:
# Description
import numpy as np

## Aggregated

In [16]:
# Treatment and pre-post variables

df_clean.loc[(df_clean["year"] >= 2010), "post_2009"] = 1
df_clean.loc[(df_clean["year"] < 2007), "post_2009"] = 0

df_clean["Treated"] = 0
df_clean.loc[(df_clean["sex"] == "female"), "Treated"] = 1

In [17]:
# Divide into urban/rural

df_urban = df_clean.loc[df_clean["region"] == "urban"].copy()
df_rural = df_clean.loc[df_clean["region"] == "rural"].copy()
df_female = df_clean.loc[(df_clean["sex"] == "female") & (df_clean["region"] == "rural")].copy()

In [18]:
df_rural.sample(3)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year,post_2009,Treated
3624,224,224,male,Umer Kot,rural,43,34,0.790698,2013,1.0,0
3262,318,318,female,Kharan,rural,515,281,0.545631,2012,1.0,1
4042,170,170,male,Rawalpindi,rural,142,131,0.922535,2014,1.0,0


In [19]:
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

mod_aggregated = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_clean).fit()
mod_aggregated.get_robustcov_results(cov_type="HC3").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     195.3
Date:                Wed, 06 Apr 2022   Prob (F-statistic):          4.15e-118
Time:                        19:27:39   Log-Likelihood:                 925.84
No. Observations:                3944   AIC:                            -1844.
Df Residuals:                    3940   BIC:                            -1819.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.6850      0.009     78.824      0.000       0.668       0.702
C(post_2009)[T.1.0]                     0.0425      0.009      4.514      0.000       0.024       0.061
C(Treated)[T.1]                        -0.1726      0.015    -11.313      0.000      -0.203      -0.143
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0397      0.017      2.385      0.017       0.007       0.072
==============================================================================
Omnibus:                      133.934   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              137.788
Skew:                          -0.432   Prob(JB):                     1.20e-30
Kurtosis:                       2.696   Cond. No.                         12.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

When we look at all rural and urban areas of all districts in Pakistan, the enrollment rate increases for girls *4 percentage points* more than for boys.
We also see that for boys, the rate of enrollment in schools declines after the years we are studying.

**LIMITATIONS:**
* We are only observing 3 year prior the Taliban campaign
* We are fitting a linear regression model to a relationship that may not be linear. We can consider the effect the taliban campaign is multifacetic and has many long term and short term effect that interact...
* Would be nice to cluster by region(rural vs urban) bu we dont have complete data of urban and rural 

## Rural Districts only

In [20]:
# RURAL

mod_rural = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_rural).fit()
mod_rural.get_robustcov_results(cov_type="HC3").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     139.9
Date:                Wed, 06 Apr 2022   Prob (F-statistic):           6.83e-83
Time:                        19:27:40   Log-Likelihood:                 393.14
No. Observations:                2178   AIC:                            -778.3
Df Residuals:                    2174   BIC:                            -755.5
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.6644      0.010     66.734      0.000       0.645       0.684
C(post_2009)[T.1.0]                     0.0437      0.011      3.822      0.000       0.021       0.066
C(Treated)[T.1]                        -0.1966      0.017    -11.708      0.000      -0.230      -0.164
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0418      0.020      2.131      0.033       0.003       0.080
==============================================================================
Omnibus:                       61.446   Durbin-Watson:                   1.661
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.308
Skew:                          -0.207   Prob(JB):                     1.77e-09
Kurtosis:                       2.478   Cond. No.                         9.76
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

Not too different at least in terms of the regression. Although, noticed a few things:
* Larger standard error

## Cities controlled by terrorist in 2009 vs not
(only measuring differences in women)

In [21]:
taliabn_dominance = [
    "South Waziristan",
    "North Waziristan",
    "Orakzai",
    "Kurram",
    "Khyber",
    "Mohmand",
    "Bajur",
    "Darra Adamkhel",
    "Swat",
    "Upper Dir",
    "Lower Dir",
    "Bannu",
    "Lakki Marwat",
    "Tank",
    "Peshawar",
    "Dera Ismail Khan",
    "Mardan",
    "Charsadda",
    "Kohat",
]

In [22]:
# New treatment variables

df_female["Treated"] = 0
df_female.loc[(df_female["subprovince"].isin(taliabn_dominance)), "Treated"] = 1
df_female.sample(5)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year,post_2009,Treated
3716,316,316,female,Karachi,rural,127,107,0.842520,2013,1.0,0
383,383,383,female,Upper Dir,rural,67,19,0.283582,2004,0.0,1
4252,380,380,female,Nowshera,rural,83,64,0.771084,2014,1.0,0
3390,446,446,female,Washuk,rural,521,146,0.280230,2012,1.0,0
1058,342,342,female,Nawabshah,rural,486,167,0.343621,2006,0.0,0


In [30]:
from linearmodels import PanelOLS
df_for_panelols = df_female.set_index(["subprovince", "year"])

mod = PanelOLS.from_formula(
    "rate_enrollment ~ Treated * post_2009 + EntityEffects + TimeEffects",
    data=df_for_panelols,
    drop_absorbed=True,
).fit()
mod.summary

/Users/clarissaache/opt/miniconda3/lib/python3.9/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/Users/clarissaache/opt/miniconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Treated, post_2009

  warnings.warn(


Dep. Variable:,rate_enrollment,R-squared:,0.0043
Estimator:,PanelOLS,R-squared (Between):,0.0143
No. Observations:,1089,R-squared (Within):,0.0104
Date:,"Thu, Apr 07 2022",R-squared (Overall):,0.0120
Time:,15:25:23,Log-likelihood,1027.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.0878
Entities:,130,P-value,0.0435
Avg Obs:,8.3769,Distribution:,"F(1,948)"
Min Obs:,1.0000,,
Max Obs:,19.000,F-statistic (robust):,4.0878


In [32]:
print(mod.summary.as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}     &  rate\_enrollment  & \textbf{  R-squared:         }   &      0.0043      \\
\textbf{Estimator:}         &      PanelOLS      & \textbf{  R-squared (Between):}  &      0.0143      \\
\textbf{No. Observations:}  &        1089        & \textbf{  R-squared (Within):}   &      0.0104      \\
\textbf{Date:}              &  Thu, Apr 07 2022  & \textbf{  R-squared (Overall):}  &      0.0120      \\
\textbf{Time:}              &      15:25:23      & \textbf{  Log-likelihood     }   &      1027.7      \\
\textbf{Cov. Estimator:}    &     Unadjusted     & \textbf{                     }   &                  \\
\textbf{}                   &                    & \textbf{  F-statistic:       }   &      4.0878      \\
\textbf{Entities:}          &        130         & \textbf{  P-value            }   &      0.0435      \\
\textbf{Avg Obs:}           &       8.3769       & \textbf{  Distribution:      }   &     F(1,948)     \\


In [26]:
# Taliban proximity

mod_proximity = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_female).fit()
mod_proximity.get_robustcov_results(cov_type="HC3").summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     15.88
Date:                Wed, 06 Apr 2022   Prob (F-statistic):           4.14e-10
Time:                        19:29:00   Log-Likelihood:                 30.367
No. Observations:                1089   AIC:                            -52.73
Df Residuals:                    1085   BIC:                            -32.76
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.4742      0.015     31.862      0.000       0.445       0.503
C(post_2009)[T.1.0]                     0.0823      0.017      4.711      0.000       0.048       0.117
C(Treated)[T.1]                        -0.0584      0.027     -2.188      0.029      -0.111      -0.006
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0322      0.035      0.926      0.354      -0.036       0.100
==============================================================================
Omnibus:                      230.948   Durbin-Watson:                   1.650
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               45.917
Skew:                          -0.067   Prob(JB):                     1.07e-10
Kurtosis:                       2.003   Cond. No.                         12.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [27]:
print(mod_proximity.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                       & rate\_enrollment & \textbf{  R-squared:         } &     0.027   \\
\textbf{Model:}                               &       OLS        & \textbf{  Adj. R-squared:    } &     0.024   \\
\textbf{Method:}                              &  Least Squares   & \textbf{  F-statistic:       } &     9.993   \\
\textbf{Date:}                                & Wed, 06 Apr 2022 & \textbf{  Prob (F-statistic):} &  1.69e-06   \\
\textbf{Time:}                                &     20:36:29     & \textbf{  Log-Likelihood:    } &    30.367   \\
\textbf{No. Observations:}                    &        1089      & \textbf{  AIC:               } &    -52.73   \\
\textbf{Df Residuals:}                        &        1085      & \textbf{  BIC:               } &    -32.76   \\
\textbf{Df Model:}                            &           3      & \textbf{                     } &             \\
\textbf{Covariance Type:}         